In [ ]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20

# Loading data

Pandas has several function for loading data, two of the most commonly used are `read_excel` and `read_csv` that respectively read data from excel files or data from a plain text file. 

Use `pd.read_excel` to read the sheet *Cellscreens* from `datafiles/Cellscreens.xls`

Use `pd.read_csv` to read the data from `datafiles/Cellscreens.csv` and `datafiles/Cellscreens.tsv`. Can you load both files correctly? If not, compare the two files (note that Jupyter can open text files) and look through the documentation of `pd.read_csv` to find the correction options.
    * 

The columns in the `.tsv` file are separated by tabs (`\t`) while in `.csv` comma's are used. By default, `pd.read_csv` expects the separator, also called the *delimiter*, to be a comma, but this can be changed with the *delimiter* argument:

In [ ]:
pd_csv = pd.read_csv('datafiles/Cellscreens.csv')
pd_tsv = pd.read_csv('datafiles/Cellscreens.tsv',delimiter='\t')
display(pd_csv)
display(pd_tsv)

# Writing data 

A DataFrame can be exported with the function `DataFrame.to_csv()`, where `DataFrame` is replaced with the name of the DataFrame you want to export. 

Use the documentation to save one of the DataFrames from the previous exercise to a comma separated text names `datafiles/out.txt`. 

Have a look at the file, do you notice the clutter? By default, Pandas adds the row numbers, which just messes up the output. Use the documentation to create a similar file without row numbers:

Export the same data using *tab* as a delimiter.

# Data curation

Sometimes you will have to manually curate your dataframe for things that did not load in correctly. For this we will first mess up the existing data frame, and then we make it nice again. But, before we do anything, we create a copy so that we do not really mess up the data:

In [ ]:
ndf = df.copy()

Next, we generate some fake data of the same length as our dataframe:

In [ ]:
fake = np.zeros(len(ndf))
fake[10] = np.nan
fake[20] = -1
fake[30] = np.nan

A new column can be added to the data frame as follows: `ndf[name] = value`. Where `value` is either a single value  which is then assigned to each row, or a list or numpy array with the same length as the dataframe. Add a column 'Fake data' to `ndf` and set the values to `fake`:

In the fake data there are some missing value, which are set to `np.nan` (nan = not a number). With `ndf.fillna(replace)` you can change these values; set them to 0:

Running `fillna` directly on `ndf` returns a new dataframe where all the *not a number* entries are replace, but it does not assign that new dataframe to ndf, therefore we must assign it (`ndf = ...`); do this:

Fake data is not a very smart name. We can rename a column name with `ndf.rename(columns={old_name : new_name})`. Use this to make *fake data*, *real data*. Print the columns (`ndf.columns`) of `ndf` to check whether it worked (note that `rename` works similar to `fillna`)

Finally, let's just remove this column, which can be done with the `ndf.drop([list of column names],axis=1)`. Use this to remove the *real data* and use `ndf.columns` to test if it worked as expected.

# Data selection

In [ ]:
df = pd.read_csv('datafiles/Cellscreens.tsv',delimiter='\t')

You can display a DataFrame with the `display` function. 

In [ ]:
display(df)

## Columns

However, you may not want to see, and use, all data. There are several ways to select data. First of all, you can select columns. For this, it can be practical to have a list of all columns:

In [ ]:
df.columns

You can access a column with:

In [ ]:
df['Compound_name']

Print the contents of the 2nd column ('Disease') without explicitly using the column name:

In the example above we only see two values: BRCA and GBM. Using `df[column].unique()`, we can print all unique values in a column. Use this function to see all unique values in the Disease column.

In [ ]:
df[df.columns[1]].unique()

Find out how many unique values each column has. Remember that you can get the length of a list with `len(list)`

## Select rows based on values

You can select rows based on the values of certain columns:

In [ ]:
df[df['Disease']=='BRCA']

You can combine multiple conditions, e.g.:
* and: `df[(df['x']=='y')&(df['y']=='x')]`
* or: `df[(df['x']=='y')|(df['y']=='x')]`

Select the rows for the following conditions:
* Patients who received Tramadol
* Patients with BRCA who received Tramadol
* Patients with GBM who recieved Codein or Fentanyl
* Patients with a pain rating larger than 6
* Patients with a pain rating larger than 6 that received the maximum painkiller dose

Use what you learned on selecting columns and rows to identify which compound, Tramadol or Fentanyl, results in lower pain ratings:

For this we want to know the pain ratings for Tramadol or Fentanyl:

Since these are more than 50 values per compound, we use `unique` to find the unique values:

Clearly, Fentanyl works much better than Tramadol

# Using data

You can use mathematical operation on columns of a DataFrame

In [ ]:
2*df['Pain_rating']

Create a copy of the DataFrame (`new = df.copy()`) and convert the compound concentration from $\mu$M to mM:

DataFrame columns can also be converted into numpy arrays that can be used for computations:

In [ ]:
x = df['Pain_rating'].values
2*x

Compute the mean pain rating of BRCA patients

Because computing statistics like the mean are commonly used, this is build into Pandas:

In [ ]:
df.mean()

Note that Pandas only computes the mean over columns with numerical values. 

You can also get the mean for a specific column:

In [ ]:
df['Pain_rating'].mean()

or

In [ ]:
df.mean()['Pain_rating']

Note that there are similar functions, such as `df.std`, `df.var`, `df.min`, `df.sum`.

Use what you have learned so far to answer the following questions:

### Which disease, BRCA or GBM, results in the most pain?

### Which disease, BRCA or GBM, results in the highest pain rating?


### Which medication works best

# Grouping



To find out which medicine works best you had to select to rows for each compound and then compute the mean, there is actually an easier way to do this:

In [ ]:
df.groupby('Compound_name').mean()['Pain_rating']

The function `groupby` can be applied on any Pandas DataFrame and only works in combination with an operator like `mean`. To help you understand how `groupby` works, we will go through its workings step by step. When you call `groupby` in a DataFrame `df` for column `Compound_name`, a bin is created for each unique value in the column and then the row numbers with matching column values are put into those bins.

In [ ]:
df.groupby('Compound_name').groups

When instead of one column, a list of columns is used, there will be a bin for each unique combination of values from the used columns:

In [ ]:
df.groupby(['Compound_name','Disease']).groups

When you have the grouping you want, you can perform operations on all groups, for example:

In [ ]:
df.groupby('Compound_name').mean()

But we are not interested in the mean concentration, so we next only request the `Pain_rating` column:

In [ ]:
df.groupby('Compound_name').mean()['Pain_rating']

Use the `groupby` function to see if the performance of the drugs differs between diseases.